A notebook used to run experiments

In [ ]:
from scripts.nca import VoxelUtil as util
from scripts.vox.Vox import Vox

seed_1 = util.custom_seed(_plus_x='red', _plus_y='green', _plus_z='blue')
Vox().load_from_tensor(seed_1).render(_show_grid=True)

seed_2 = util.custom_seed(_minus_x='green', _plus_y='red', _plus_z='blue')
Vox().load_from_tensor(seed_2).render(_show_grid=True)

seed_3 = util.custom_seed(_minus_x='red', _minus_y='green', _plus_z='blue')
Vox().load_from_tensor(seed_3).render(_show_grid=True)

seed_4 = util.custom_seed(_plus_x='green', _minus_y='red', _plus_z='blue')
Vox().load_from_tensor(seed_4).render(_show_grid=True)

In [8]:
x = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
print (x[:-3])
print (x[-1])
print (x[-2])
print (x[-3])
print (x[-1:])
print (x[-2:])
print (x[-3:])
print (x[-1:])
print (x[-2:-1])
print (x[-3:-2])

[0, 1, 2, 3, 4, 5, 6]
9
8
7
[9]
[8, 9]
[7, 8, 9]
[9]
[8]
[7]
